In [1]:
%store

Stored variables and their in-db values:
__importClassification             -> IPython.macro.Macro(u"# Handle table-like data and
__importRegression                 -> IPython.macro.Macro(u"# Handle table-like data and


In [2]:
%store -r __importRegression

In [3]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
endMonth = 29
fnameTest = '../input/validation/test_' + str(endMonth) + '.csv'
fnameTrain = '../input/validation/train_' + str(endMonth) + '.csv'
train = pd.read_csv(fnameTrain)
test = pd.read_csv(fnameTest)
#train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')
items = pd.read_csv("../input/items.csv")
categories = pd.read_csv("../input/item_categories.csv")

print (train.shape, test.shape, items.shape, categories.shape)

((2664552, 6), (31909, 3), (22170, 3), (84, 2))


In [5]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [6]:
test.head()

,shop_id,item_id,item_cnt_day
0,2,486,2.0
1,2,792,1.0
2,2,949,1.0
3,2,950,1.0
4,2,1090,1.0


## Extract shop total sales

In [7]:
train_shop = train.copy()

In [8]:
train_shop.drop(['date', 'date_block_num', 'item_price', 'item_id'], axis=1, inplace=True)

In [9]:
train_shop.head()

,shop_id,item_cnt_day
0,59,1.0
1,25,1.0
2,25,-1.0
3,25,1.0
4,25,1.0


In [10]:
train_shop = train_shop.groupby(['shop_id'], as_index=False).agg({'item_cnt_day':sum})

In [11]:
train_shop.head()

,shop_id,item_cnt_day
0,0,11705.0
1,1,6311.0
2,2,26540.0
3,3,25124.0
4,4,39797.0


In [12]:
train_shop['shop_total_sales'] = train_shop.item_cnt_day

In [13]:
train_shop.drop('item_cnt_day', axis=1, inplace=True)

In [14]:
train_shop.head()

,shop_id,shop_total_sales
0,0,11705.0
1,1,6311.0
2,2,26540.0
3,3,25124.0
4,4,39797.0


## Extract item total sales

In [15]:
train_items = train.copy()

In [16]:
train_items.drop(['date', 'date_block_num', 'shop_id', 'item_price'], axis=1, inplace=True)

In [17]:
train_items.head()

,item_id,item_cnt_day
0,22154,1.0
1,2552,1.0
2,2552,-1.0
3,2554,1.0
4,2555,1.0


In [18]:
train_items = train_items.groupby(['item_id'], as_index=False).agg({'item_cnt_day':sum})

In [19]:
train_items['item_total_sales'] = train_items.item_cnt_day
train_items.drop('item_cnt_day', axis=1, inplace=True)

In [20]:
train_items.head()

,item_id,item_total_sales
0,0,1.0
1,1,6.0
2,2,2.0
3,3,2.0
4,4,1.0


In [21]:
train.pop('date')
train.pop('date_block_num')
train.pop('item_price');

In [22]:
train.head()

,shop_id,item_id,item_cnt_day
0,59,22154,1.0
1,25,2552,1.0
2,25,2552,-1.0
3,25,2554,1.0
4,25,2555,1.0


In [23]:
train = train.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day':sum})

In [24]:
print train.shape
train.head()

(396546, 3)


,shop_id,item_id,item_cnt_day
0,0,30,31.0
1,0,31,11.0
2,0,32,16.0
3,0,33,6.0
4,0,35,15.0


In [25]:
train.item_cnt_day.describe()

count    396546.000000
mean          8.349415
std          59.686950
min         -12.000000
25%           1.000000
50%           3.000000
75%           7.000000
max       17678.000000
Name: item_cnt_day, dtype: float64

Take average sales in 33 months and remove duplicates

In [26]:
train.item_cnt_day = train.item_cnt_day / (endMonth-1)

In [27]:
train.head()

,shop_id,item_id,item_cnt_day
0,0,30,1.107143
1,0,31,0.392857
2,0,32,0.571429
3,0,33,0.214286
4,0,35,0.535714


In [28]:
train.drop_duplicates(inplace=True)
train.shape

(396546, 3)

In [29]:
print test.shape
test.head()

(31909, 3)


,shop_id,item_id,item_cnt_day
0,2,486,2.0
1,2,792,1.0
2,2,949,1.0
3,2,950,1.0
4,2,1090,1.0


## Merge shop_total_sales and item_total_sales

In [30]:
train = pd.merge(train, train_shop, on='shop_id', sort=False, how='left')
test = pd.merge(test, train_shop, on='shop_id', sort=False, how='left')

train['shop_total_sales'] = train['shop_id'].map(train_shop)
test['shop_total_sales'] = test['shop_id'].map(train_shop)

In [31]:
train = pd.merge(train, train_items, on='item_id', sort=False, how='left')
test = pd.merge(test, train_items, on='item_id', sort=False, how='left')

train['item_total_sales'] = train['item_id'].map(train_shop)
test['item_total_sales'] = test['item_id'].map(train_shop)

In [32]:
train.head()

,shop_id,item_id,item_cnt_day,shop_total_sales,item_total_sales
0,0,30,1.107143,11705.0,2071.0
1,0,31,0.392857,11705.0,1339.0
2,0,32,0.571429,11705.0,1974.0
3,0,33,0.214286,11705.0,764.0
4,0,35,0.535714,11705.0,222.0


In [33]:
test.head()

,shop_id,item_id,item_cnt_day,shop_total_sales,item_total_sales
0,2,486,2.0,26540.0,1213.0
1,2,792,1.0,26540.0,382.0
2,2,949,1.0,26540.0,343.0
3,2,950,1.0,26540.0,471.0
4,2,1090,1.0,26540.0,NaN


In [34]:
test.fillna(0)

,shop_id,item_id,item_cnt_day,shop_total_sales,item_total_sales
0,2,486,2.0,26540.0,1213.0
1,2,792,1.0,26540.0,382.0
2,2,949,1.0,26540.0,343.0
3,2,950,1.0,26540.0,471.0
4,2,1090,1.0,26540.0,0.0
5,2,1099,1.0,26540.0,337.0
6,2,1409,1.0,26540.0,87.0
7,2,1464,1.0,26540.0,25.0
8,2,1485,3.0,26540.0,646.0
9,2,1496,1.0,26540.0,811.0


In [35]:
print (train.shape, test.shape)

((396546, 5), (31909, 5))


In [36]:
train_labels = train.pop('item_cnt_day')

In [37]:
labels = test.pop('item_cnt_day')

test_id = test.pop('ID')

# LightGBM tunning

LBC = LGBMRegressor()
lb_param_grid = {
              'n_estimators' : [10, 50, 100],
              'learning_rate': [0.1, 0.01],
              'max_depth': [4, 8]
              }

lGBC = GridSearchCV(LBC,param_grid = lb_param_grid, cv=5,
                     scoring="neg_mean_squared_error", n_jobs= -1, verbose = 1)

lGBC.fit(train, train_labels)

lgbm = lGBC.best_estimator_

# Best score
lGBC.best_score_

In [38]:
lgbm = LGBMRegressor()
lgbm.fit(train, train_labels)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
       objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [39]:
predictions = lgbm.predict(test)
predictions.shape

(31909,)

In [40]:
labels.shape

(31909,)

Clipping predictions in the [0, 20] range

In [41]:
for i in range(len(predictions)):
    if predictions[i] > 20:
        predictions[i] = 20
    if predictions[i] < 0:
        predictions[i] = 20

for i in range(len(labels)):
    if labels[i] > 20:
        labels[i] = 20
    if labels[i] < 0:
        labels[i] = 20

Compute RMSE

In [42]:
np.sqrt(mean_squared_error(labels, predictions))

5.6889934998307607

## Submission

test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': predictions})
print test_submit.shape
test_submit.to_csv('lgbm_all_train_totalSales.csv', index=False)
test_submit.head()

test_submit.item_cnt_month.describe()